In [1]:
import ee
import pandas as pd

In [ ]:
ee.Authenticate()
ee.Initialize()

In [3]:
def HS2_col(aoi, date_min, date_max):
    """
       Input : Area of Interest, Date Range
       Output : Harmonized Sentinel-2 Image
    """
    S2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
                    .filterBounds(aoi)\
                    .filterDate(ee.Date(date_min), ee.Date(date_max))\
                    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))# Filters images so that the % cloud cover is atmost equal to CLOUD_FILTER
    
    S2_cloud_prb = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')\
                    .filterBounds(aoi)\
                    .filterDate(ee.Date(date_min), ee.Date(date_max))
    
    """Connects both ImageCollection and links them together by each images respective counterpart through the 'system:index' property 
       and labels the matched collections as s2cloudless"""
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': S2,
        'secondary': S2_cloud_prb,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [4]:
def get_coords(lon, lat, size):
    """
       Input : Longitutde, Latitude, Size
       Output : size*size region centered at the given longitude and latitude coordinates (Area of Interest)
    """
    coords = [
         [lon - size/2., lat - size/2.],
         [lon + size/2., lat - size/2.],
         [lon + size/2., lat + size/2.],
         [lon - size/2., lat + size/2.],
         [lon - size/2., lat - size/2.]
    ]
    aoi = ee.Geometry.Polygon(coords)
    
    return aoi

In [5]:
def add_cloud_bands(img):
    """
       Input : Image
       Output : Image with cld_prb and is_cloud added as image bands
    """
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability') # Obtains s2cloudless image with the probability band
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds') # Identifies pixels as clouds if value of 'probability' is higher than CLD_PRB_THRESH
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [6]:
def add_shadow_bands(img):
    """
       Input : Image
       Output : Image with dark_pixels, cld_proj, shadows added as image bands
    """
    not_water = img.select('SCL').neq(6) #Identifies water pixels from the SCL band
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels') #Identifies potential shadow cloud pixels that are not water pixels
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE'))) # Determine the direction to project cloud shadow from clouds (assumes UTM projection)
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10) # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))
    shadows = cld_proj.multiply(dark_pixels).rename('shadows') # Identify the intersection of dark pixels with cloud shadow projection
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [7]:
def add_cld_shdw_mask(img):
    """
       Input : Image
       Output : Image with final cloud masking band
    """
    img_cloud = add_cloud_bands(img)
    img_cloud_shadow = add_shadow_bands(img_cloud)
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0) # Combines cloud and cloud shadow mask, values of clouds and shadows set as equal to 1, and others to 0
    "Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input 20 m scale is for speed, and assumes clouds don't require 10 m precision"
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))
    return img.addBands(is_cld_shdw)

In [8]:
def apply_cld_shdw_mask(img):
    """
       Input : Image
       Output : Cloud-Free Image
    """
    not_cld_shdw = img.select('cloudmask').Not() # Invert cloudmask band, clouds and shadows become 0, everything else 1
    return img.select('B.*').updateMask(not_cld_shdw) # Update surface reflectance bands with cloud masking

In [9]:
def download_S2(img, aoi, filename):
    """
       Input : Image, Area of Interest, Filename
       Output : Image downloaded as filename.tif in Google Drive
    """
    
    task = ee.batch.Export.image.toDrive(**{
        'image' : img,
        'description' : filename,
        'scale' : 10,
        'region' : aoi,
        'crs' : 'EPSG:4326',
        'folder' : 'BOD5/0.15Deg'
    })
    task.start()

In [10]:
"""
    Credits to Google jdbcode for Sentinel-2 Cloud Masking
    Sentinel-2 Cloud Masking with s2cloudless 
"""
#Values in these sections can be changed to further optimize cloud masking Sentinel 2 Images
CLOUD_FILTER = 60 # Maximum image cloud cover percent allowed in image collection
CLD_PRB_THRESH = 65 # Cloud probability (%); values greater than are considered cloud
NIR_DRK_THRESH = 0.15 #Near-infrared reflectance; values less than are considered potential cloud shadow
CLD_PRJ_DIST = 2 # Maximum distance (km) to search for cloud shadows from cloud edges
BUFFER = 50 # Distance (m) to dilate the edge of cloud-identified objects

In [11]:
#For reading large CSV GRQA files
const_date = "2017-03-28"
cols = ['lat_wgs84', 'lon_wgs84', 'obs_date', 'site_id', 'param_code', 'obs_value', 'obs_iqr_outlier']
iter_csv = pd.read_csv('BOD5_GRQA.csv', iterator=True, chunksize=1000, sep=';', usecols=cols)
#Only reads data after 2017-03-28 and are not outliers
df = pd.concat([chunk[(chunk['obs_date'] > const_date) & (chunk['obs_iqr_outlier'] == 'no')] for chunk in iter_csv])

#df = pd.read_csv('Water_Data.csv')
site_id = df['site_id'].values
lat = df['lat_wgs84'].values
lon = df['lon_wgs84'].values
date_min = df['obs_date'].values
obs_value = df['obs_value'].values
param_code = df['param_code'].values

# Adds 1 month to each date so S2 images are downloaded and compiled within a 1 month period
date_max = []
for date in date_min:
    new_date = pd.to_datetime(date) + pd.DateOffset(months=1)
    new_date = new_date.date()
    date_max.append(new_date.strftime('%Y-%m-%d'))

In [15]:
# For-loop that runs the code using the arrays that stored the data read from the CSV file
size = 0.15
for i in range(0, len(site_id)):
    aoi = get_coords(lon[i], lat[i], size)
    img_col = HS2_col(aoi, date_min[i], date_max[i])
    CFS2 = img_col.map(add_cld_shdw_mask)\
                  .map(apply_cld_shdw_mask)\
                  .select('B4','B3','B2','B1','B5','B6','B7','B8','B8A','B9','B11','B12')\
                  .median()
    #filename = {0};{1};BOD5{2}'.format(site_id[c], date_min[c], obs_value[c])
    #download_S2(CFS2, aoi, filename)
    filename = f'{site_id[i]}_{date_min[i]}_{param_code[i]}_{obs_value[i]}'
    download_S2(CFS2, aoi, filename)